In [1]:
import dpkt
import datetime
from dpkt.utils import mac_to_str, inet_to_str

In [2]:
def print_packets(pcap):
    """Print out information about each packet in a pcap
       Args:
           pcap: dpkt pcap reader object (dpkt.pcap.Reader)
    """
    # For each packet in the pcap process the contents
    for timestamp, buf in pcap:

        # Print out the timestamp in UTC
        print('Timestamp: ', str(datetime.datetime.utcfromtimestamp(timestamp)))

        # Unpack the Ethernet frame (mac src/dst, ethertype)
        eth = dpkt.ethernet.Ethernet(buf)
        print('Ethernet Frame: ', mac_to_str(eth.src), mac_to_str(eth.dst), eth.type)

        # Make sure the Ethernet data contains an IP packet
        if not isinstance(eth.data, dpkt.ip.IP):
            print('Non IP Packet type not supported %s\n' % eth.data.__class__.__name__)
            continue

        # Now access the data within the Ethernet frame (the IP packet)
        # Pulling out src, dst, length, fragment info, TTL, and Protocol
        ip = eth.data

        # Print out the info, including the fragment flags and offset
        print('IP: %s -> %s   (len=%d ttl=%d DF=%d MF=%d offset=%d)\n' %
              (inet_to_str(ip.src), inet_to_str(ip.dst), ip.len, ip.ttl, ip.df, ip.mf, ip.offset))

    # Pretty print the last packet
    print('** Pretty print demo **\n')
    eth.pprint()


def test():
    """Open up a test pcap file and print out the packets"""
    with open('data/session.pcap', 'rb') as f:
        pcap = dpkt.pcap.Reader(f)
        print_packets(pcap)


if __name__ == '__main__':
    test()

Timestamp:  2022-06-29 07:50:56.197048
Ethernet Frame:  02:42:ac:11:00:02 02:42:ac:11:00:03 2048
IP: 172.16.0.1 -> 172.27.26.101   (len=60 ttl=64 DF=1 MF=0 offset=0)

Timestamp:  2022-06-29 07:50:56.197111
Ethernet Frame:  02:42:ac:11:00:03 02:42:ac:11:00:02 2048
IP: 172.27.26.101 -> 172.16.0.1   (len=60 ttl=64 DF=1 MF=0 offset=0)

Timestamp:  2022-06-29 07:50:56.197122
Ethernet Frame:  02:42:ac:11:00:02 02:42:ac:11:00:03 2048
IP: 172.16.0.1 -> 172.27.26.101   (len=52 ttl=64 DF=1 MF=0 offset=0)

Timestamp:  2022-06-29 07:50:56.202782
Ethernet Frame:  02:42:ac:11:00:02 02:42:ac:11:00:03 2048
IP: 172.16.0.1 -> 172.27.26.101   (len=569 ttl=64 DF=1 MF=0 offset=0)

Timestamp:  2022-06-29 07:50:56.202835
Ethernet Frame:  02:42:ac:11:00:03 02:42:ac:11:00:02 2048
IP: 172.27.26.101 -> 172.16.0.1   (len=52 ttl=64 DF=1 MF=0 offset=0)

Timestamp:  2022-06-29 07:50:56.203026
Ethernet Frame:  02:42:ac:11:00:03 02:42:ac:11:00:02 2048
IP: 172.27.26.101 -> 172.16.0.1   (len=1465 ttl=64 DF=1 MF=0 offset